# My attempt to move issues between project boards
## References:
https://developer.github.com/v3/guides/getting-started/

https://developer.github.com/v3/

In [ ]:
import requests
import json

In [ ]:
sess = requests.Session()

In [ ]:
sess.headers['Authorization'] = 'token <personal token here>'
sess.headers['Accept'] = 'application/vnd.github.inertia-preview+json'  # to use beta project api

In [ ]:
sess.headers

In [ ]:
res = sess.get('https://api.github.com/user')
res.ok

In [ ]:
user_info = res.json()
user_info

In [ ]:
# List projects (org-level projects who up using "/repos/" api:
res = sess.get('https://api.github.com/repos/ksbeattie/api_fun/projects')
res.ok

In [ ]:
res.json()

In [ ]:
# A list of project dictionaries
projs = res.json()
len(projs)

In [ ]:
for proj in projs:
    print('name: {name:20} #: {number:<3} id: {id}'.format(**proj))

In [ ]:
# Get the columns from the project
proj = projs[1]
res = sess.get(proj['columns_url'])
res.ok

In [ ]:
res.json()

In [ ]:
# List the columns
cols = res.json()
for col in cols:
    print('column: {name:20} #: id: {id}'.format(**col))

In [ ]:
# Get the cards from the 0th column
col = cols[0]
res = sess.get(col['cards_url'])
res.ok

In [ ]:
# list the cards
cards = res.json()
for card in cards:
    print('card: {id}'.format(**card))
    print('  {}'.format(card.get('content_url', card['note'])))

In [ ]:
# Get the 0th card
card = cards[2]
card

In [ ]:
# A card is either a note (where 'note' is not None) or an issue (with a 'content_url')
if card['note']:
    print('note only: "{}"'.format(card['note']))  # We're done digging down in the API
else:
    res = sess.get(card['content_url'])
    print('Issue: {}'.format(res.ok))

In [ ]:
issue = res.json()
issue

In [ ]:
# An issue is either just an issue (no more digging) or a PR (with a 'pull_request')
if 'pull_request' in issue.keys():
    res = sess.get(issue['pull_request']['url'])
    print('PR: {}'.format(res.ok))
    pullr = res.json()
else:
    print('Issue only: "{}"'.format(issue['body']))

In [ ]:
pullr

In [ ]:
# Create a new note card
card_info = {'note': 'This is a new note'}
res = sess.post('https://api.github.com/projects/columns/3722510/cards',
                data=json.dumps(card_info)
               )
res.ok

In [ ]:
res.json()

In [ ]:
# Create pr
res = sess.get('https://api.github.com/repos/ksbeattie/api_fun/pulls/3')
res.ok

In [ ]:
res.json()

In [ ]:
res = sess.delete(card['url'][:-1])
res.ok

In [ ]:
res.content